In [11]:
# Packages
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm
import yfinance as yf

# Dataset
# Set start and end date
start_date = '2022-04-01'
end_date = '2024-03-31'

# Code of the Index or Stocks
code = ['CL=F', 'ES=F', 'GC=F', 'NG=F', 'NQ=F', '^GSPC', '^IXIC', 'YM=F', 'RTY=F']

# Download data
df = yf.download(code, start_date, end_date)
df = df['Adj Close']

# Daily return daily prices and remove any NA
logreturn = np.log(df[code]/df[code].shift(1))
logretur = logret.dropna()

[*********************100%%**********************]  9 of 9 completed


In [13]:
# Function of expected return
def e_return(weight, logreturn):
    return np.sum(logreturn.mean()*weight)

In [15]:
# Function of standard deviation
def stdev_return(weight, cov_matrix):
    variance = weight.T @cov_matrix @weight
    return np.sqrt(variance)

In [17]:
cov_matrix = logreturn.cov()
print(cov_matrix)

NameError: name 'logreturn' is not defined

In [ ]:
# Weight
port_value = 100000
# weight = np.array([1/9]*9) # Equal
# weight = np.array([0.1, 0.1, 0.1, 0.1, 0.2, 0.1, 0.1, 0.1, 0.1]) # varcov
weight = np.array([0.00458493, 0.18084041, 0.27384339, 0.0021879,  0.2060439,  0.03673825, 0.10243592, 0.170245,   0.0230803]) # MC
port_e_return = e_return(weight, logreturn)
port_stdev_return = stdev_return(weight, cov_matrix)
print(weight)

In [ ]:
# Calculate scenario GainLoss
days = 10
def scenario_gain_loss(port_value, port_stdev_return, days):
    zscore = np.random.normal(0, 1)
    return port_value*port_e_return*days + port_value*port_stdev_return *zscore*np.sqrt(days)

In [ ]:
# Monte Carlo Simulation
num_sim = 100000
scenario_return = []

for i in range(num_sim):
    scenario_return.append(scenario_gain_loss(port_value, port_stdev_return, days))

In [ ]:
# Confidence interval and VaR
alpha = [0.01, 0.05, 0.10]
VaR_01 = -np.percentile(scenario_return, 100 *alpha[0])
VaR_05 = -np.percentile(scenario_return, 100 *alpha[1])
VaR_10 = -np.percentile(scenario_return, 100 *alpha[2])
print(VaR_10)
print(VaR_05)
print(VaR_01)

In [ ]:
# Plot the results
plt.hist(scenario_return, bins=50, density=True, color='y')
plt.xlabel('Gain/Loss ($)')
plt.ylabel('Frequency')
plt.title(f'Distribution of Portfolio Gain/Loss Over {days} Days')
plt.axvline(VaR_10, color='r', linestyle='dashed', linewidth=2, label=f'VaR at {alpha[2]:.0%} confidence level')
plt.axvline(VaR_05, color='g', linestyle='dashed', linewidth=2, label=f'VaR at {alpha[1]:.0%} confidence level')
plt.axvline(VaR_01, color='b', linestyle='dashed', linewidth=2, label=f'VaR at {alpha[0]:.0%} confidence level')
plt.legend()
plt.show()